# "News Sentiment and Stock Market Data Exploration"
> "Performing Exploratory Data Analysis for the Saudi Stock Market"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [data-wrangling, EDA]
- image: images/arabic-stock-market-sentiment-analysis/featured_sectors_bar.png
- hide: false
- search_exclude: true

# News Sentiment and Stock Market Data Exploration

In [ ]:
#!pip install pandas-profiling[notebook,html]
#!pip install cufflinks
#!pip install chart_studio
#!pip install googletrans

In [ ]:
#plotly.offline doesn't push your charts to the clouds
import plotly.offline as pyo
#allows us to create the Data and Figure objects
from plotly.graph_objs import *
#plotly.plotly pushes your charts to the cloud  
# import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.express as px

import matplotlib.pyplot as plt
%matplotlib inline

# import cufflings to easily plot pandas data frames
import cufflinks as cf
# work with cufflinks offline and set its theme
import plotly.io as pio
pio.templates.default = "plotly_white"

cf.go_offline()
cf.set_config_file(theme='white')

import pandas as pd
from pandas_profiling import ProfileReport
from googletrans import Translator
import datetime as dt

import numpy as np

import os
import json
import re
import sys

In [ ]:
print(pd.__version__)

In [ ]:
def print_full(x):
    pd.set_option('display.max_colwidth', None)
    return x

def reset():
    pd.reset_option('display.max_colwidth')

In [ ]:
cf.getThemes()

In [ ]:
cf.colors.scales() 

In [ ]:
df = pd.read_excel('Samira - Arabic Stock Market Data.xlsx')

In [ ]:
df.info()

In [ ]:
df['Polarity'].value_counts()

We need to normalize this column

In [ ]:
df['Polarity'] = df['Polarity'].str.strip().str.capitalize()

In [ ]:
df['Polarity'].value_counts()

In [ ]:
df.head(2)

In [ ]:
translator = Translator()

unique_sectors = df['Sector'].unique().tolist()

unique_sectors_en = [translator.translate(sector, src='ar').text.title() for sector in unique_sectors]

print(unique_sectors)
print(unique_sectors_en)

In [ ]:
unique_sectors_en = ['Banks', 'Tasi Main Market Index', 
                     'Basic Materials', 'Capital Goods', 
                     'Long-Term Goods', 'Retail Of Luxury Goods', 
                     'Consumer Services', 'Commercial And Professional Services', 
                     'Energy', 'Food Production', 'Pharmaceutical', 
                     'Public Utility', 'Investment And Financing', 'Transport', 
                     'Healthcare', 'Food Segmentation', 'Real Estate M & D', 
                     'Media And Entertainment', 'Real Estate Traded Funds', 
                     'Telecommunications', 'Applications And Technology Services', 'Insurance']


sector_map = {sector_ar:sector_en for sector_ar, sector_en in zip(unique_sectors, unique_sectors_en)}

In [ ]:
df['Sector_en'] = df['Sector'].map(sector_map)

In [ ]:
df.groupby(['Sector', 'Sector_en']).size()

In [ ]:
drop_cols = ['Date', 'Company ID']

In [ ]:
df.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# df['Date'] = pd.to_datetime(df['Date_1st_Additional'] + ' ' + df['Time'])

In [ ]:
df['Date'] = df.apply(lambda r : pd.datetime.combine(r['Date_1st_Additional'], r['Time']), axis=1)

In [ ]:
df.drop(['Date_1st_Additional'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.columns = ['The Quantity Handled' if col=='The quantity Handled' else col for col in df.columns]

In [ ]:
df.info()

In [ ]:
# profile = ProfileReport(df, title="EDA Report", explorative=True)
# profile.to_widgets()
# profile.to_file("EDA-Report.html")

## Data Cleaning and Normalization

### Exploring Nans

In [ ]:
df.isnull().sum()      # nans in each column

In [ ]:
# df[df['News Details'].isnull()]

In [ ]:
# remove NANs
df = df.dropna(subset=['News Details'])

In [ ]:
df.isna().sum()

### Exploring Duplicates

In [ ]:
# check number of duplicated records (rows)
df.duplicated().sum()

In [ ]:
# check the duplicated reviews
print_full(df.loc[df['News Details'].duplicated()])

On close examination of the duplicated rows, it turned out they're not fully duplicated. So we'll leave them as is 

### Storing the Clean Data for Later Use

In [ ]:
df.to_csv('data/clean_data.csv', index=False)

In [ ]:
reset()

In [ ]:
df.head(2)

## Descriptive Statistics

In [ ]:
cats = ['Sector', 'Company Name', 'Polarity']
nums = ['Oppening', 'The Highest', 'The Lowest', 'Closing', 'Changing', 'Changing%', 
        'The Quantity Handled', 'Total Current Value (SAR)', 'Number of Deals']

texts = ['News Title', 'News Details', 'Polarity Words']

date = ['Date']

#### Numerical Variables Statistics

**Note:** The huge numbers in the column `Total Current Value` is due to the huge amount itself. it's not an approximation

In [ ]:
df[nums].describe().round(2)

In [ ]:
def describe_cat(cat_var, n=5):
    if cat_var == 'Date':
        col = df[cat_var].dt.date
    else:
        col = df[cat_var]
        
    unique = len(col.unique())
    top_counts = col.value_counts().to_frame().reset_index().head(n)
    top_counts_str = "، ".join([f'{level}: {count}' for level,count in top_counts.values])
    
    summary = pd.Series({'Variable': cat_var, 'Unique': unique, 'Top Counts': top_counts_str})
    
    return summary
    
    
def describe_cats(cat_vars):
    summary = pd.DataFrame(np.nan, columns=['Variable', 'Unique', 'Top Counts'], index=range(len(cat_vars)))
    
    for i, cat in enumerate(cat_vars):
        summary.iloc[i] = describe_cat(cat)
    
    summary['Unique'] = summary['Unique'].astype('int')
        
    return summary

In [ ]:
describe_cat('Date')

In [ ]:
print_full(describe_cats(cats+date))

In [ ]:
reset()

## Exploratory Data Analysis

### Exploring Singe Categorical Variables

#### The Polarity

In [ ]:
# !pip install notebook ipywidgets

In [ ]:
from IPython.display import HTML

In [ ]:
fig = (  df['Polarity']
       .value_counts()
       .reset_index()
       .pipe(px.pie,
             values='Polarity', names='index',
             title='Polarity Distribution',
             color='Polarity',
             color_discrete_sequence=px.colors.qualitative.Prism,
             hole=0.1)
       .update_traces(textinfo='percent+label+value')
    
)

HTML(fig.to_html(include_plotlyjs='cdn'))

#### Most Featured Sectors

In [ ]:
n_most=10

In [ ]:
(df
 .groupby('Sector_en')
 .size()
 .to_frame(name='Count')
 .reset_index()
 .sort_values('Count', ascending=False)
 .head(n_most)
 .pipe(px.bar,
       x='Sector_en', y='Count', 
#        color='Count',
#        color_continuous_scale=px.colors.sequential.GnBu,
#        color_continuous_scale=px.colors.sequential.Blues,
#        color_continuous_scale=px.colors.sequential.Plasma_r,
       text='Count', title='Most Featured Sectors', labels={'Count': 'No. Articles', 'Sector_en':'Sector'}))

In [ ]:
featured_sectors = (df['Sector_en']
                     .value_counts()
                     .head(n_most)
                     .index.tolist())
featured_sectors

#### Most Featured Companies

In [ ]:
n_most = 10

(df['Company Name']
 .value_counts()
 .head(n_most)
 .iplot(kind='bar', color='navy', yTitle='# of News Articles', title=f'Most {n_most} Featured Companies'))

In [ ]:
featured_companies = (df['Company Name']
                     .value_counts()
                     .head(n_most)
                     .index.tolist())
featured_companies

### Exploring Interaction Between 2 Categorical Variables

#### Polarity & Sector

##### Method (1) Grouped Barplot

In [ ]:
polarity_by_sector = (df
                      .groupby(['Sector_en', 'Polarity'])
                      .size()
                      .unstack()
                      .fillna(0)
                      .sort_values('Positive', ascending=False))

polarity_by_sector

In [ ]:
polarity_by_sector.loc[featured_sectors]

In [ ]:
(polarity_by_sector.loc[featured_sectors]
 .iplot(kind='bar',
        colorscale='-gnbu',#'set1',
        layout=dict(title='Distribution of Polarity over Sectors',
                    yaxis=dict(title='# of Articles'),
                    xaxis=dict(title='Sector'))))

In [ ]:
(polarity_by_sector.loc[featured_sectors]
 .reset_index()
 .melt(id_vars='Sector_en', value_name='Count')
 .pipe(
     px.bar,
     x='Sector_en', y='Count', color='Polarity', text='Count',
     barmode='group', opacity=0.8,
     title='Distribution of Polarity over Sectors',
     labels={'Count': 'No. of Articles'},
     color_discrete_sequence = px.colors.diverging.Spectral_r,
#      color_discrete_sequence=px.colors.sequential.Plasma,
#      color_discrete_sequence=px.colors.qualitative.Prism,
#      color_discrete_sequence=px.colors.sequential.GnBu_r,
#      color_discrete_sequence=px.colors.qualitative.Set1
))

##### Method (2) Heatmap

In [ ]:
(polarity_by_sector.loc[featured_sectors]
 .iplot(kind='heatmap',
        title='Distribution of Polarity over Sectors',
        colorscale='gnbu'))

In [ ]:
(polarity_by_sector.loc[featured_sectors]
 .T
 .pipe(px.imshow,
       color_continuous_scale=px.colors.sequential.GnBu,
       title='Distribution of Polarity over Sector'))

#### Polarity & Company

##### Method (1) Groupded Barplot

In [ ]:
polarity_by_company = (df
                      .groupby(['Company Name', 'Polarity'])
                      .size()
                      .unstack()
                      .fillna(0)
                      .sort_values('Positive', ascending=False))

polarity_by_company

In [ ]:
(polarity_by_company.loc[featured_companies]
 .iplot(kind='bar',
        colorscale='blues',
        layout=dict(title='Distribution of Polarity over Companies',
                    yaxis=dict(title='# of Articles'),
                    xaxis=dict(title='Company'))))

##### Method (2) Heatmap

In [ ]:
(polarity_by_company.loc[featured_companies]
 .iplot(kind='heatmap',
        title='Distribution of Polarity over Companies',
        colorscale='blues'))

### Exploring Interaction Between 3 Categorical Variables

In [ ]:
df_featured = df[(df['Company Name'].isin(featured_companies)) & (df['Sector_en'].isin(featured_sectors))]

In [ ]:
px.parallel_categories(df_featured, width=1000, height=700, title='Parallel Plot of Sector, Company, Polarity',
                       labels={'Sector_en': 'Sector'})

### Exploring Single Numerical Variables

In [ ]:
df['Closing'].iplot(kind='box', 
                    boxpoints='outliers',
                    color='blue',
                    title='Distribution of Closing Price')

In [ ]:
df.pipe(px.box,
        y='Closing',
        points='outliers',
        title='Distribution of Closing Price')

In [ ]:
df['Changing%'].iplot(kind='box', 
                      boxpoints='outliers',
                      color='purple',
                      title='Distribution of Changing%')

In [ ]:
(df
 .pipe(px.box,
       y='Changing%'))

### Qustion: How wast the variable `Changing%` computed?

In [ ]:
df['change_pct'] = df['Closing'].pct_change()

In [ ]:
df[['Changing%', 'change_pct']]

### Exploring Relationship Between Numerical Variables

In [ ]:
df.columns

In [ ]:
(df[nums]
 .corr()
 .fillna(0)
 .round(2))

In [ ]:
(df[nums]
 .corr()
 .iplot(kind='heatmap',
        colorscale='ylgnbu',#'rdylbu',#,rdbu, 
        layout=dict(title='Correlation Between All Numerical Variables',
                    width=700,
                    margin=dict(b=120))))

#### Number of Deals vs. Changing

In [ ]:
(df.fillna(0)
 .pipe(px.scatter,
       size='Total Current Value (SAR)',
       size_max=70,
       x='Number of Deals',
       y='Changing',
       title='<b>Number of Deals vs. Changing in Price</b> <br>size deontes <i>Total Current Value</i>',
       range_y=[-20, 20],
       range_x=[0, 25000]))

In [ ]:
fig = (df.fillna(0)
         .pipe(px.scatter,
               x='Closing',
               y='Changing',
               color='Polarity',
               size='Total Current Value (SAR)',
               size_max=60,
               marginal_x='histogram',
               color_discrete_sequence = px.colors.diverging.Spectral_r,
#                color_discrete_sequence=px.colors.sequential.GnBu,
#                color_discrete_sequence=['red', 'blue', 'green'],
               range_y=[-15, 15],
               range_x=[0, 200],
               height=600,
               facet_col='Polarity',
               title='<b>Closing Price vs. Changing in Price Across Polarities</b><br>size deontes <i>Total Current Value</i>'))

fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

In [ ]:
fig = (df.fillna(0)
        .pipe(px.scatter,
              x='Closing',
              y='Changing%',
              color='Polarity',
              size='Total Current Value (SAR)',
              size_max=60,
              marginal_x='histogram',
             color_discrete_sequence = px.colors.diverging.Spectral_r,
#             color_discrete_sequence=px.colors.sequential.GnBu_r,
#             color_discrete_sequence = px.colors.diverging.Spectral_r,
#             color_discrete_sequence=['red', 'blue', 'green'],
             range_y=[-20, 20],
             range_x=[0, 300],
             height=600,
             facet_col='Polarity',
             title='<b>Closing Price vs. Changing Percent in Price</b><br>size deontes <i>Total Current Value</i>'))

fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

### Exploring Interaction Between Numerical and Categorical

In [ ]:
returns_by_polarity_stats = (df.reset_index()
                             .pivot(columns='Polarity', values='Changing%')).describe()

returns_by_polarity_stats

In [ ]:
median = (df[['Polarity', 'Changing%']].reset_index()
          .pivot(columns='Polarity', values='Changing%')
          .apply(lambda x: x.median()))

median.name = 'median'

In [ ]:
returns_by_polarity_stats.append(median)

In [ ]:
(df.reset_index()
 .pivot(columns='Polarity', values='Changing%')
 .iplot(kind='box', boxpoints='outliers',
        colorscale='gnbu',
        layout=dict(title='Distribution of Percent Change Over Polarity')))

In [ ]:
(df
 .pipe(px.box,
       x='Polarity',
       y='Changing%',
       color='Polarity',
       color_discrete_sequence = px.colors.diverging.Spectral_r,
       points='outliers',
       title='Distribution of Percent Change Over Polarity'))

# fig.update_layout(showlegend=False)

From the Boxplot above we can't deduce a strong correlation between the polarity of a news article and a change in stock price. 

This should be corroborated by a statistical test, though!

### What's Happening over Time? 

#### Exploring the Date Variable

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
df = df.sort_index()

In [ ]:
date_hist = dict(
    data=[
        Histogram(
            x=df.index,
        marker=dict(
            opacity=0.9,
            line=dict(
                color='white', width=1
            )
         )
      )
    ],
    layout=dict(
        title='Distribution of No. of Articles over Months',
        xaxis=dict(
            range=[df.index.min() - pd.Timedelta(days=90), df.index.max()]
        )
    )
)

pyo.iplot(date_hist)

## The Following is Additional Extra Work

#### Moving Avg. of Numerical Variables 

In [ ]:
company_name = 'شركة أبناء عبدالله عبدالمحسن الخضري'
window = 10
num_var = 'Closing'

fig = (df
        .loc[df['Company Name'] == company_name, num_var]
        .sort_index()
        .rolling(window).mean()
        .iplot(mode='lines', color='blue', size=8, asFigure=True, 
               title=f'{window} Days Moving Avg. ({num_var}) <br> {company_name}'))

In [ ]:
fig

In [ ]:
company_name = 'شركة أبناء عبدالله عبدالمحسن الخضري'

fig_1 = (df
 .loc[df['Company Name'] == company_name]
 .groupby('Polarity')
 .resample('A')
 .size()
 .T
 .iplot(kind='bar', barmode='group',
        colorscale='gnbu',
        title=f'Polarity Evolution over Time <br> {company_name}',
        asFigure=True))

In [ ]:
fig_1

In [ ]:
fig_1['layout']['title']['text']

In [ ]:
fig_1.add_trace(fig['data'][0])


## Market Index

We can create an index for the market by choosing the largest company in each sector, in terms of `Total Current Value`. Then plot the overall index market cap over time overlayed on the sentiment as above

In [ ]:

index_comps = df.loc['2019'].set_index('Company Name').groupby('Sector_en')['Total Current Value (SAR)'].nlargest(1)

index_comps = index_comps.reset_index().sort_values('Total Current Value (SAR)', ascending=False)

index_comps

In [ ]:
index_comps_names = index_comps['Company Name']; index_comps_names

In [ ]:
df_index = df[df['Company Name'].isin(index_comps_names)]

In [ ]:
index_series = (df_index[['Company Name', 'Total Current Value (SAR)']]
                 .pivot(columns='Company Name', values='Total Current Value (SAR)')
                 .sum(axis=1)
                 .loc[: 'Nov 2019'])

In [ ]:
(df_index[['Company Name', 'Total Current Value (SAR)']]
 .pivot(columns='Company Name', values='Total Current Value (SAR)')
 .sum(axis=1)
 .loc['2011': 'Nov 2019']
 .rolling(180).mean()
 .iplot(color='blue',
        layout=dict(title='30-days Moving Average of Market Index')))

In [ ]:
index_series.iplot()

In [ ]:
(df_index
 .groupby('Polarity')
 .resample('A')
 .size()
 .T
 .iplot(kind='bar', title='Annual Polarity Distribution for the Market Index'))

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

In [ ]:
with open('ArSEL1.0.txt') as f:
    text = f.read()
    text = re.sub('###', ';', text)


In [ ]:
print(text[:2000])

In [ ]:
with open('ArSEL1.0_modified.txt', 'w') as f:
    f.write(text)

In [ ]:
cols = 'AWN_OFFSET;EWN_OFFSET;POS_Tag;AWN_Lemma;SAMA_Lemma;Pos_Sentiment_score;Neg_Sentiment_Score;Confidence;AFRAID;AMUSED;ANGRY;ANNOYED;DONT_CARE;HAPPY;INSPIRED;SAD'.split(';')

In [ ]:
cols

In [ ]:
lex = pd.read_table('ArSEL1.0_modified.txt', sep=';', skiprows=20, header=None)
lex.columns = cols

In [ ]:
lex.head()

In [ ]:
lex.info()

In [ ]:
lex['SAMA_Lemma'].value_counts()

### How to Read Facebook's Fasttext pretrained Models into Gensim

read this ASAP: <a href='https://radimrehurek.com/gensim/models/fasttext.html' target='_blank'>fasttext loading into gensim</a>